In [54]:
import pandas as pd
from openpyxl import workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import numpy as np
import os

### Loading file to dataframes

Loading .xlsx file created in agesex.ipynb

In [55]:
# Creating ordered list of DFs to name each sheet coming in from workbook
df_names = ['DF_total_all','DF_male_all','DF_female_all','DF_total_whi',
            'DF_total_male_whi','DF_total_female_whi','DF_total_baa',
            'DF_total_male_baa','DF_total_female_baa','DF_total_aian',
            'DF_total_male_aian','DF_total_female_aian','DF_total_aa',
            'DF_total_male_aa','DF_total_female_aa','DF_total_nhop',
            'DF_total_male_nhop','DF_total_female_nhop','DF_total_sor',
            'DF_total_male_sor','DF_total_female_sor','DF_total_tom',
            'DF_total_male_tom','DF_total_female_tom','DF_total_hol',
            'DF_total_male_hol','DF_total_female_hol'
            ]

In [56]:
# Dictionary to store the dataframes
dfs = {}

# Get the path of the current working directory
cwd = os.getcwd()

# Construct the full path to the .xlsx file
file_path = os.path.join(cwd, '2020_agesex_data.xlsx')

# Load each sheet of the .xlsx file into a named dataframe
for name in df_names:
    df = pd.read_excel(file_path, sheet_name=name, header=None)  # Specify header=None to treat the first row as data
    df.columns = df.iloc[0]  # Set the first row as the column headers
    df = df[1:]  # Exclude the first row from the data
    df.reset_index(drop=True, inplace=True)  # Reset the index
    dfs[name] = df

### Verifying dataframes load in

In [57]:
# Print the names of the dataframes in 'dfs'
print("Data Frames in 'dfs':")
for name in dfs.keys():
    print(name)

Data Frames in 'dfs':
DF_total_all
DF_male_all
DF_female_all
DF_total_whi
DF_total_male_whi
DF_total_female_whi
DF_total_baa
DF_total_male_baa
DF_total_female_baa
DF_total_aian
DF_total_male_aian
DF_total_female_aian
DF_total_aa
DF_total_male_aa
DF_total_female_aa
DF_total_nhop
DF_total_male_nhop
DF_total_female_nhop
DF_total_sor
DF_total_male_sor
DF_total_female_sor
DF_total_tom
DF_total_male_tom
DF_total_female_tom
DF_total_hol
DF_total_male_hol
DF_total_female_hol


In [58]:
dfs['DF_male_all'].dtypes

0
Total                  object
Under 1 Year           object
1 Year                 object
2 Years                object
3 Years                object
                        ...  
110  Years and Over    object
Location               object
State                  object
County                 object
FIPS                   object
Length: 108, dtype: object

In [59]:
df = dfs['DF_male_all']
# Print the columns
print("Columns of DF_male_all:")
for column in df.columns:
    print(column)

Columns of DF_male_all:
Total
Under 1 Year
1 Year
2 Years
3 Years
4 Years
5 Years
6 Years
7 Years
8 Years
9 Years
10 Years
11 Years
12 Years
13 Years
14 Years
15 Years
16 Years
17 Years
18 Years
19 Years
20 Years
21 Years
22 Years
23 Years
24 Years
25 Years
26 Years
27 Years
28 Years
29 Years
30 Years
31 Years
32 Years
33 Years
34 Years
35 Years
36 Years
37 Years
38 Years
39 Years
40 Years
41 Years
42 Years
43 Years
44 Years
45 Years
46 Years
47 Years
48 Years
49 Years
50 Years
51 Years
52 Years
53 Years
54 Years
55 Years
56 Years
57 Years
58 Years
59 Years
60 Years
61 Years
62 Years
63 Years
64 Years
65 Years
66 Years
67 Years
68 Years
69 Years
70 Years
71 Years
72 Years
73 Years
74 Years
75 Years
76 Years
77 Years
78 Years
79 Years
80 Years
81 Years
82 Years
83 Years
84 Years
85 Years
86 Years
87 Years
88 Years
89 Years
90 Years
91 Years
92 Years
93 Years
94 Years
95 Years
96 Years
97  Years
98  Years
99  Years
100 to 104  Years
105 to 109  Years
110  Years and Over
Location
State
Co

### Building weighted average column

Grouping Dataframes with like structures

In [60]:
stats_df = ['DF_male_all','DF_female_all','DF_total_male_whi',
             'DF_total_female_whi','DF_total_male_baa','DF_total_female_baa',
             'DF_total_male_aian','DF_total_female_aian','DF_total_male_aa',
             'DF_total_female_aa','DF_total_male_nhop','DF_total_female_nhop',
             'DF_total_male_sor','DF_total_female_sor','DF_total_male_tom',
             'DF_total_female_tom','DF_total_male_hol','DF_total_female_hol'
             ]

Setting new column names and columns to be typed to integer

In [63]:
col_names = ['Total','0','1','2','3','4','5','6','7','8','9','10','11','12',
             '13','14','15','16','17','18','19','20','21','22','23','24','25',
             '26','27','28','29','30','31','32','33','34','35','36','37','38',
             '39','40','41','42','43','44','45','46','47','48','49','50','51',
             '52','53','54','55','56','57','58','59','60','61','62','63','64',
             '65','66','67','68','69','70','71','72','73','74','75','76','77',
             '78','79','80','81','82','83','84','85','86','87','88','89','90',
             '91','92','93','94','95','96','97','98','99','102','107','110',
             'Location','State','County','FIPS'
             ]

col_ints = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14',
            '15','16','17','18','19','20','21','22','23','24','25','26','27',
            '28','29','30','31','32','33','34','35','36','37','38','39','40',
            '41','42','43','44','45','46','47','48','49','50','51','52','53',
            '54','55','56','57','58','59','60','61','62','63','64','65','66',
            '67','68','69','70','71','72','73','74','75','76','77','78','79',
            '80','81','82','83','84','85','86','87','88','89','90','91','92',
            '93','94','95','96','97','98','99','102','107','110'
            ]

Iterating through grouped dataframes to change column name and type

In [65]:
# Iterate over the dataframes in 'dfs'
for key, df in dfs.items():
    # Check if the dataframe is present in 'stats_df'
    if key in stats_df:
        # Modify column names based on 'col_names' list
        df.columns = col_names[:len(df.columns)]
        
        # Loop through the columns to convert to integers
        for col in df.columns:
            if col in col_ints:
                # Change the column name to an integer
                df.rename(columns={col: int(col)}, inplace=True)

In [ ]:
        # Calculate weighted average for each row
        weights = df[col_ints].columns.astype(int)
        values = df[col_ints].values
        df['Average Age'] = (values * weights).sum(axis=1) / weights.sum()

### Changing column types

In [22]:
# Convert specific columns to str
str_columns = ['Location', 'State', 'County', 'FIPS']

for column in str_columns:
    df[column] = df[column].astype(str)

# Convert remaining columns to int
int_columns = [col for col in df.columns if col not in str_columns]
df[int_columns] = df[int_columns].astype(int, errors='ignore')

In [ ]:
dfs['DF_male_all'].dtypes

### Building Percent columns

In [33]:
stats_df = ['DF_male_all','DF_female_all','DF_total_male_whi',
             'DF_total_female_whi','DF_total_male_baa','DF_total_female_baa',
             'DF_total_male_aian','DF_total_female_aian','DF_total_male_aa',
             'DF_total_female_aa','DF_total_male_nhop','DF_total_female_nhop',
             'DF_total_male_sor','DF_total_female_sor','DF_total_male_tom',
             'DF_total_female_tom','DF_total_male_hol','DF_total_female_hol'
             ]

perc_col = ['Under 1 Year','1 Year','2 Years','3 Years','4 Years',
             '5 Years','6 Years','7 Years','8 Years','9 Years','10 Years',
             '11 Years','12 Years','13 Years','14 Years','15 Years','16 Years',
             '17 Years','18 Years','19 Years','20 Years','21 Years','22 Years',
             '23 Years','24 Years','25 Years','26 Years','27 Years','28 Years',
             '29 Years','30 Years','31 Years','32 Years','33 Years','34 Years',
             '35 Years','36 Years','37 Years','38 Years','39 Years','40 Years',
             '41 Years','42 Years','43 Years','44 Years','45 Years','46 Years',
             '47 Years','48 Years','49 Years','50 Years','51 Years','52 Years',
             '53 Years','54 Years','55 Years','56 Years','57 Years','58 Years',
             '59 Years','60 Years','61 Years','62 Years','63 Years','64 Years',
             '65 Years','66 Years','67 Years','68 Years','69 Years','70 Years',
             '71 Years','72 Years','73 Years','74 Years','75 Years','76 Years',
             '77 Years','78 Years','79 Years','80 Years','81 Years','82 Years',
             '83 Years','84 Years','85 Years','86 Years','87 Years','88 Years',
             '89 Years','90 Years','91 Years','92 Years','93 Years','94 Years',
             '95 Years','96 Years','97  Years','98  Years','99  Years',
             '100 to 104  Years','105 to 109  Years','110  Years and Over',
             ]

#### Building percent to total

In [34]:
# Calculate the percentage for each column in perc_col
for col in perc_col:
    percent_col = col + ' Percent to Total'
    new_col_index = df.columns.get_loc(col) + 1  # Get the index of the current column and add 1
    df.insert(new_col_index, percent_col, (df[col] / df['Total']) * 100)
    df[percent_col] = df[percent_col].round(2)

ValueError: cannot insert Under 1 Year Percent to Total, already exists

In [53]:
print(dfs['DF_male_all'].values)

[[40783 437 1.07 ... '029' '21029' 507.2429934041145]
 [6205 70 1.13 ... '031' '21031' 78.55745366639806]
 [6136 72 1.17 ... '033' '21033' 75.40600032610467]
 ...
 [19687 190 0.97 ... '227' '48227' 260.4909053957931]
 [1684 20 1.19 ... '229' '48229' 25.907902554961378]
 [49310 593 1.2 ... '231' '48231' 612.1351154017767]]


In [35]:
df = dfs['DF_male_all']
# Print the columns
print("Columns of DF_male_all:")
for column in df.columns:
    print(column)

Columns of DF_male_all:
Total
Under 1 Year
Under 1 Year Percent to Total
1 Year
1 Year Percent to Total
2 Years
2 Years Percent to Total
3 Years
3 Years Percent to Total
4 Years
4 Years Percent to Total
5 Years
5 Years Percent to Total
6 Years
6 Years Percent to Total
7 Years
7 Years Percent to Total
8 Years
8 Years Percent to Total
9 Years
9 Years Percent to Total
10 Years
10 Years Percent to Total
11 Years
11 Years Percent to Total
12 Years
12 Years Percent to Total
13 Years
13 Years Percent to Total
14 Years
14 Years Percent to Total
15 Years
15 Years Percent to Total
16 Years
16 Years Percent to Total
17 Years
17 Years Percent to Total
18 Years
18 Years Percent to Total
19 Years
19 Years Percent to Total
20 Years
20 Years Percent to Total
21 Years
21 Years Percent to Total
22 Years
22 Years Percent to Total
23 Years
23 Years Percent to Total
24 Years
24 Years Percent to Total
25 Years
25 Years Percent to Total
26 Years
26 Years Percent to Total
27 Years
27 Years Percent to Total
28

In [48]:
weight = [12,26,30,17,22,12,36,17,25,14,26,41,30,29,51,30,21,34,14,23,21,11,8,
         17,4,7,12,23,15,22,11,4,25,18,10,25,28,3,18,17,14,25,10,14,10,12,7,5,
         17,9,9,11,18,15,11,7,7,9,4,6,7,2,5,1,7,0,0,8,2,3,3,8,2,6,4,0,3,3,1,1,
         2,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0
         ]

col_names = ['Total','0','1','2','3','4','5','6','7','8','9','10','11','12',
             '13','14','15','16','17','18','19','20','21','22','23','24','25',
             '26','27','28','29','30','31','32','33','34','35','36','37','38',
             '39','40','41','42','43','44','45','46','47','48','49','50','51',
             '52','53','54','55','56','57','58','59','60','61','62','63','64',
             '65','66','67','68','69','70','71','72','73','74','75','76','77',
             '78','79','80','81','82','83','84','85','86','87','88','89','90',
             '91','92','93','94','95','96','97','98','99','102','107','110',
             'Location','State','County','FIPS'
             ]

print(np.average(a=value, weights=weight))

27.02
